In [80]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
# Import des modules nécessaires
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

# Création de plusieurs wallets pour les nœuds de la blockchain
walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

# Création d'un wallet utilisateur (ex: Alice)
walletAlice = Wallet()

# Initialisation du mécanisme de Proof of Stake avec le premier nœud
pos = ProofOfStake(walletNode1.publicKey)

#On reset le network
Node.reset_network()

#On connecte les 3 noeuds au network
node1 = BlockchainNode(walletNode1,pos)
node2 = BlockchainNode(walletNode2,pos)
node3 = BlockchainNode(walletNode3,pos)

assert len(Node.get_all_nodes())==3
"Success"

'Success'

In [82]:
# Exemple de source de code pour le smart contract
source_code = """
class MyNFTContract:
    def __init__(self, issuerPublicKey):
        self.issuerPublicKey = issuerPublicKey
        self.nfts = {}  # Dictionnaire des NFTs

    def mint(self, ownerPublicKey, nftId):
        if nftId not in self.nfts:
            self.nfts[nftId] = ownerPublicKey
            print(f"NFT {nftId} frappé par {ownerPublicKey}")
        else:
            raise ValueError(f"NFT {nftId} déjà existant")

    def transfer(self, senderPublicKey, receiverPublicKey, nftId):
        if nftId in self.nfts and self.nfts[nftId] == senderPublicKey:
            self.nfts[nftId] = receiverPublicKey
            print(f"NFT {nftId} transféré de {senderPublicKey} à {receiverPublicKey}")
        else:
            raise ValueError("NFT inexistant ou propriété invalide")
"""

# Création du Smart Contract (définition)
contract_definition = SmartContractDefinition(issuerPublicKey=walletAlice.publicKey, sourceCode=source_code, Ntoken=5)

# Signer le certificat
walletAlice.sign(contract_definition)

# Instanciation du Smart Contract
contract_instance = contract_definition.instantiate_contract()

# Ajouter le Smart Contract au premier nœud (node1)
node1.new_certificate(contract_definition) # Ajouter le smart contract au nœud

print(node1.blockchain.blockList)
print(node2.blockchain.blockList)
print(node3.blockchain.blockList)

Vérification de la légitimité du certificat :
Clé publique de l'émetteur : 30820122300d06092a864886f70d01010105000382010f003082010a0282010100d104bb98ba76813d2456eca0ec9a010df014561a21e31b84f1a9dc3215e2b32343c7696c3120fc608b64818e2bd6a047f4a409015e61e4ec89478faa58fff88321321c55afe8b5048d89ec68a7a2b274701d15f1c66306e1ec0da87b058d79e41a80ea568a8d6d27ddd629f02a000748ea9a22808ce12cf3ce6d256926da6dc15e486acb2197a8d738c46317e46326cfc5e446201b9cf23000692a3e7408e357d891634ce7f116e9551fc8c0b61a8e296cb4061cc530a7f6e27c87d261dbb591f9930f7345daf286a8aedd7b524b8fab2498949606fb034fa6209f8b7ebe50ceabd48958e92ca07f22afad1bc9fe743dc50d47316b4aa3cb7eeee16763e9e8c50203010001
Signature : 793ec6470d16c51a84574d07a2c95b0359f6862a129ce4a9425c27b9c034d6473d93cad347e1dc08b178fd3a8271b075260ab0c4c5ba6c0645ec98e1c9ad84d5d0c4c11a693ce864b2383da3e3f5772670b6b51efc5c34e1ae52dcc429efbf7ea3f4ac2a747cddebb32b761e32b62a135cc8b8347fab9234810e845f8eb020035e62fb7a7822ceefdafd38af1aa6a2cbe85f17d4c86c180bd9dea6f96c5d6e795e7a

In [71]:
# Frapper un NFT pour Alice via node1
contract_definition.mint(ownerPublicKey="Alice", nftId="NFT1")

NFT NFT1 frappée par Alice


In [72]:
print(contract_definition.owners)

{'NFT1': 'Alice'}


In [75]:
#Transferer un NFT de Alice à Edouard
contract_definition.transfer(senderPublicKey="Alice", receiverPublicKey="Edouard", nftId="NFT1")
print(contract_definition.owners)

NFT NFT1 transférée de Alice à Edouard
{'NFT1': 'Edouard'}
